In [44]:
#Importando bibliotecas que iremos utilizar

from datetime import datetime 
from googleapiclient.discovery import build 
import pandas as pd

In [2]:
youTubeApiKey = 'SUA_API_AQUI' #Definindo a chave da API
youtube = build('youtube', 'v3', developerKey=youTubeApiKey) #Parâmetros da API

In [3]:
channel_id = 'UCcS5ak6sn8uPx2aCmDeGkpA' #Definindo o ID do canal que queremos coletar

#Neste exemplo, usarei o ID do canal do Insper

In [4]:
channel_details = youtube.channels().list(id=channel_id,
                                          part='statistics').execute() 

#Obtendo dados gerais de estatísticas do canal. Útil para determinar quantos vídeos o canal tem.

channel_details

{'kind': 'youtube#channelListResponse',
 'etag': 'P0p2o0M5fTzyXEHdUD7DzEEQ4eo',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'vHTNwcS96RwaFTBFrLqXAL4ri2Y',
   'id': 'UCcS5ak6sn8uPx2aCmDeGkpA',
   'statistics': {'viewCount': '13063206',
    'subscriberCount': '21300',
    'hiddenSubscriberCount': False,
    'videoCount': '1152'}}]}

In [5]:
uploads = youtube.channels().list(id=channel_id,
                              part='contentDetails').execute() #Obtendo os dados da playlist 'uploads' do canal, que contém todos os seus vídeos

playlist_id = uploads['items'][0]['contentDetails']['relatedPlaylists']['uploads'] #Filtrando a consulta só com ID da playlist uploads

videos = [] #Criando uma lista vazia para armazenar os vídeos
next_page_token = None #Definindo a variável para o token da próxima página 
                        #a API do YT só permite 50 resultados por página

while True:   #loop para iterar por cada página da consulta na API e armazenar os vídeos na lista
    all_videos = youtube.playlistItems().list(playlistId=playlist_id, 
                                       part='snippet', 
                                       maxResults=50,
                                       pageToken=next_page_token).execute()
    
    videos += all_videos['items'] 
    next_page_token = all_videos.get('nextPageToken')
    
    if next_page_token is None: #Caso não haja o token, o loop é interrompido
        break

In [33]:
videos_ids = [video['snippet']['resourceId']['videoId'] for video in videos] #Filtrando os IDs de cada video
stats = [] #Criando uma lista vazia para armazenar as estatísticas de cada ID/vídeo

for video_id in videos_ids:
    res = youtube.videos().list(part='statistics', id=video_id).execute()
    stats += res['items']

In [41]:
#Criando uma lista para cada dado que quero coletar e realizando um loop para armazenar cada item
#Para a parte 'snippet', utilizei list comprehension pois todos os campos retornam um valor, mesmo que vazios

channel_title = [video['snippet']['channelTitle'] for video in videos] 
videos_title = [video['snippet']['title'] for video in videos]
url_thumbnails = [video['snippet']['thumbnails']['high']['url'] for video in videos]
published_date = [video['snippet']['publishedAt'] for video in videos]
video_description = [video['snippet']['description'] for video in videos]
videoid = [video['snippet']['resourceId']['videoId'] for video in videos]
extraction_date = [str(datetime.now())]*len(videos_ids) #Definindo o horário da extração

In [52]:
#Criando uma lista para cada dado que quero coletar e realizando um loop para armazenar cada item
#Para a parte 'statistics', foi preciso criar um try para não receber um Key Error quando um dos dados não está presente

liked = []
disliked = []
views = []
comment = []

for video in stats:
    try:
        liked.append(video['statistics']['likeCount'])
    except:
        liked.append(None)
        
for video in stats:
    try:
        disliked.append(video['statistics']['dislikeCount'])
    except:
        disliked.append(None)
        
for video in stats:
    try:
        views.append(video['statistics']['viewCount'])
    except:
        views.append(None)
        
for video in stats:
    try:
        comment.append(video['statistics']['commentCount'])
    except:
        comment.append(None)
        

In [53]:
#Criando um dataframe a partir dos dados organizados, utilizando um dicionário

df = pd.DataFrame({
    'channel' :channel_title,
    'title':videos_title,
    'video_id':videoid,
    'video_description':video_description,
    'published_date':published_date,
    'extraction_date':extraction_date,
    'likes':liked,
    'dislikes':disliked,
    'views':views,
    'comment':comment,
    'thumbnail': url_thumbnails})
df.head()

,channel,title,video_id,video_description,published_date,extraction_date,likes,dislikes,views,comment,thumbnail
0,Insper,Inovação na área de saúde,Qot6FxBIYHY,,2021-09-27T15:09:49Z,2021-10-05 20:16:11.146883,29,0,231,5,https://i.ytimg.com/vi/Qot6FxBIYHY/hqdefault.jpg
1,Insper,Encerramento da Primeira Pós-Graduação em Urba...,imxpoqwqOLc,,2021-09-17T20:42:02Z,2021-10-05 20:16:11.146883,22,0,242,1,https://i.ytimg.com/vi/imxpoqwqOLc/hqdefault.jpg
2,Insper,Oportunidades e benefícios de empreender na fa...,C3lxwg5GmYk,,2021-09-20T21:57:14Z,2021-10-05 20:16:11.146883,29,1,183,0,https://i.ytimg.com/vi/C3lxwg5GmYk/hqdefault.jpg
3,Insper,Vídeo-síntese | O futuro das pastagens e a rev...,4OunINlMfso,“O futuro das pastagens e a revolução da integ...,2021-09-23T22:04:25Z,2021-10-05 20:16:11.146883,2,0,18,0,https://i.ytimg.com/vi/4OunINlMfso/hqdefault.jpg
4,Insper,Insper Agro Global | O futuro das pastagens e ...,meGMk-BST2A,“O futuro das pastagens e a revolução da integ...,2021-09-23T21:59:07Z,2021-10-05 20:16:11.146883,6,0,30,0,https://i.ytimg.com/vi/meGMk-BST2A/hqdefault.jpg


In [57]:
df.to_csv('output_data/canal_insper.csv') #Exportando o resultado para um CSV